In [18]:
# Import required libraries
import json
import networkx as nx
from itertools import product


In [19]:
# Configuration
file_path = "../data/1eb80fb8b50.json"  # Path to the JSON file
segment_key = ["kws-l", "loc-l", "org-l", "per-l", "content-segmented"]  # Keys to extract from JSON
Years = ["2024"]  # Years to process
save_graph = True  # Whether to save the graph
name_graph_saved = "typed_graph"  # Name of the saved graph file

# Load the JSON data
try:
    with open(file_path, "r") as json_file:
        data = json.load(json_file)
except FileNotFoundError:
    print(f"File not found: {file_path}")
    data = {}
except json.JSONDecodeError as e:
    print(f"JSON decoding error: {e}")
    data = {}

# Initialize a list to store extracted data
df_kws_article = []


In [20]:
# Extract data for each article
for year in Years:
    for month in range(1, 13):
        for day in range(1, 32):
            try:
                daily_data = data['data'][str(year)][str(month)][str(day)]
                for num, data_tmp in enumerate(daily_data):
                    # Extract necessary information
                    entry = {
                        "loc-l": data_tmp.get("loc-l", []),
                        "org-l": data_tmp.get("org-l", []),
                        "per-l": data_tmp.get("per-l", []),
                        "content-segmented": data_tmp.get("content-segmented", []),
                    }
                    df_kws_article.append(entry)
            except KeyError:
                # Skip if year, month, or day doesn't exist
                continue
            except Exception as e:
                print(f"Error processing {year}-{month}-{day}: {e}")
                continue


In [21]:
# Function to determine the type of relations between entities
def determine_relation_type(entity1, entity2, content_segment):
    """
    Heuristic to type relations between two entities
    based on the content segment.
    """
    if "financial" in content_segment or "economic" in content_segment:
        return "economic"
    elif "cooperation" in content_segment or "diplomatic" in content_segment:
        return "diplomatic"
    elif "aid" in content_segment or "humanitarian" in content_segment:
        return "humanitarian"
    else:
        return "undefined"


In [ ]:
import networkx as nx

# Initialiser un graphe vide
G = nx.Graph()

for row in df_kws_article:
    # Ensure row is a dictionary (to avoid KeyError)
    if not isinstance(row, dict):
        continue

    # Extract the content-segmented data
    segments = row.get("content-segmented", [])
    
    # Ensure segments is iterable
    if not isinstance(segments, list):
        continue
    
    # Iterate over each segment to build relationships
    for segment in segments:
        entities = []
        
        # Extract entities for loc-l, org-l, and per-l
        for key in ["loc-l", "org-l", "per-l"]:
            if key in row and isinstance(row[key], list):  # Ensure the value is a list
                entities.extend(row[key])  # Append entities from the row

        # Iterate over pairs of entities to create relationships
        for entity1 in entities:
            for entity2 in entities:
                if entity1 != entity2:
                    # Ensure entities are hashable types
                    entity1 = tuple(entity1) if isinstance(entity1, list) else entity1
                    entity2 = tuple(entity2) if isinstance(entity2, list) else entity2

                    # Skip if entities are still not hashable
                    if not isinstance(entity1, (str, int, float, tuple)):
                        print(f"Skipping entity1: {entity1} (type: {type(entity1)})")
                        continue
                    if not isinstance(entity2, (str, int, float, tuple)):
                        print(f"Skipping entity2: {entity2} (type: {type(entity2)})")
                        continue
                    
                    # Determine the relation type
                    relation_type = determine_relation_type(entity1, entity2, segment)

                    # Add edges safely
                    if G.has_edge(entity1, entity2):
                        G[entity1][entity2]["weight"] += 1  # Increment weight if edge exists
                    else:
                        G.add_edge(entity1, entity2, weight=1, relation_type=relation_type)

TypeError: unhashable type: 'list'

In [ ]:
print(type(df_kws_article))


<class 'list'>


NameError: name 'entities' is not defined